In [1]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
import datetime
import geopy.distance
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'
storage = "/Volumes/easystore/Drones"
coi=['Incident No.','Description','cfs-date','Block Location','Latitude','Longitude','drone-id']
orphan_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../../data/outputs/outputs-flights-without-matching-cfs_matched-by-id.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../../data/outputs/outputs_calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

flight_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../../data/outputs/outputs_drone-seconds-block-count-per-flight.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../../data/mapping/cv-blocks.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks['GEOID20'] = cv_blocks['GEOID20'].apply(lambda x: x[3:])


Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [2]:
orphan_flight_blocks = flight_blocks[flight_blocks['id'].isin(orphan_flights['id'])]

In [3]:
calls_for_service["cfs-date"] = pd.to_datetime(calls_for_service["Date"])
orphan_flights["drone-date"] =  pd.to_datetime(orphan_flights['date'] + " " + orphan_flights['time_s'])

In [4]:
orphan_flights

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,...,Census Block Group,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code
0,32.6275301,-117.0881123,5:04pm,3f07b495a2ef58c78a6e091996d4a8aa,7-30-21,5:04pm,600 I St.,CVL63427,Domestic Violence,cvpd,...,1,060730127001003,06073012700,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
1,32.64015719,-116.99921129,4:33pm,4daa5e5a1bf0bb212fb633f367ed09a0,7-30-21,4:33pm,"Racquetball courts, 900 Otay Lakes Rd",NaN,NaN,cvpd,...,1,060730134151000,06073013415,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
2,32.63996613,-117.08269066,3:59pm,3a45193e3e1df231acb51cc49ce56a43,7-29-21,3:59pm,315 4th Ave,NaN,NaN,cvpd,...,1,060730123021013,06073012302,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
3,32.63997708,-117.08270143,1:35pm,a15ae3ca1a31bfbb4e8093c2e2abd67d,7-29-21,1:35pm,315 4th Ave,NaN,NaN,cvpd,...,1,060730123021013,06073012302,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
4,32.6148408836699,-117.080731458995,8:51am,e65d003d59263a32526d5b5796ba2897,7-29-21,8:51am,986 5th Ave,NaN,NaN,cvpd,...,4,060730131044000,06073013104,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,32.5989197,-117.0615246,2:14pm,e4fdf51abb9f2d999af03177b0e2595d,9-02-23,2:14pm,275 Orange Ave.,CVL75348,Domestic Violence,cvpd,...,2,060730132042000,06073013204,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
1949,32.6098649,-117.0813317,12:55pm,32c3272b9e976103882e5fe03b020112,9-02-23,12:55pm,Broadway/ Naples St.,CVL75330,Traffic Hazard,cvpd,...,2,060730131032011,06073013103,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
1950,32.6266906,-117.0757946,12:35pm,9d4c9e9c87b4529af56f1ff9d0117e62,9-02-23,12:35pm,720 Third Ave.,CVL75316,Subject Causing Disturbance,cvpd,...,2,060730130002000,06073013000,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
1951,32.6495326,-117.0285142,2:17am,6f373ed6fb467777e498853f18685d0c,9-02-23,2:17am,400 Milagrosa Circle,L075219,Burglary,cvpd,...,3,060730134123014,06073013412,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN


In [5]:
pc = []
for _, row in tqdm(orphan_flights.sample(frac=1).iterrows()):
    ceiling = row['drone-date'] + datetime.timedelta(minutes = 30)
    floor = row['drone-date'] - datetime.timedelta(minutes = 30)


    mask = (calls_for_service['cfs-date'] > floor) & (calls_for_service['cfs-date'] < ceiling)
    possible_calls = calls_for_service.loc[mask].copy()
    possible_calls['drone-id'] = row['id']
    possible_calls['drone-incident-id'] = row['incident_id']
    possible_calls['drone-date'] = row['drone-date']
    possible_calls['drone-type'] = row['type']
    possible_calls['drone-type'] = row['type']
    possible_calls['drone-address'] = row['address_map']
    possible_calls['drone-FIPS'] = row['Full FIPS (block)']

    # possible_calls = possible_calls[possible_calls['Full FIPS (block)']==row['Full FIPS (block)']].copy()
    
    pc.append(possible_calls)
    

0it [00:00, ?it/s]

In [6]:
possible_calls_by_time_alone = pd.concat(pc)

In [7]:
orphan_flights = orphan_flights[~orphan_flights['id'].isin(possible_calls_by_time_alone[possible_calls_by_time_alone['Latitude'].astype(str)=='0']['drone-id'])]

In [8]:
possible_cfs_by_distance = []
for _, flight in tqdm(orphan_flights.iterrows()):
#   find possible calls by time
    possible_cfs_for_flight = possible_calls_by_time_alone[possible_calls_by_time_alone['drone-id']==flight['id']]
    
    orphan_flight = orphan_flight_blocks[orphan_flight_blocks['id']==flight['id']].copy()
    orphan_flight['seconds'] = orphan_flight['seconds'].astype(int)

    lingering_blocks = orphan_flight[orphan_flight['seconds']>=60]
    lingering_blocks = cv_blocks[cv_blocks['GEOID20'].isin(lingering_blocks['GEOID20'])]
    if lingering_blocks.shape[0]>0:
        for _, cfs in possible_cfs_for_flight.iterrows():
            for _, block in lingering_blocks.iterrows():
                distance = geopy.distance.geodesic((cfs['Latitude'],cfs['Longitude']),(block['INTPTLAT20'],block['INTPTLON20'])).miles
                if str(cfs['Latitude'])=='0':
                    cfs["lingering-block"] = block['GEOID20']
                    cfs["distance"] = distance
                    possible_cfs_by_distance.append(cfs)

                elif distance <= .5:
                    cfs["lingering-block"] = block['GEOID20']
                    cfs["distance"] = distance
                    possible_cfs_by_distance.append(cfs)
                elif distance > 30:
                    cfs["lingering-block"] = block['GEOID20']
                    cfs["distance"] = distance
                    possible_cfs_by_distance.append(cfs)

    

0it [00:00, ?it/s]

In [9]:
possible_cfs_by_dist = pd.concat(possible_cfs_by_distance)
possible_cfs_by_dist = pd.DataFrame(possible_cfs_by_distance)


In [10]:
matched = possible_cfs_by_dist['drone-id']
orphan_flights_not_close = orphan_flights[~orphan_flights['id'].isin(matched)]



In [15]:
orphan_flights_not_close.shape[0]

741

In [12]:
pc = []
for _, row in tqdm(orphan_flights_not_close.iterrows()):
    ceiling = row['drone-date'] + datetime.timedelta(minutes = 30)
    floor = row['drone-date'] - datetime.timedelta(minutes = 30)

    mask = (calls_for_service['cfs-date'] > floor) & (calls_for_service['cfs-date'] < ceiling)
    possible_calls = calls_for_service.loc[mask].copy()
    possible_calls['drone-id'] = row['id']
    possible_calls['drone-incident-id'] = row['incident_id']
    possible_calls['drone-date'] = row['drone-date']
    possible_calls['drone-type'] = row['type']
    possible_calls['drone-address'] = row['address_map']
    possible_calls['drone-FIPS'] = row['Full FIPS (block)']
    
    # possible_calls = possible_calls[possible_calls['Full FIPS (block)']==row['Full FIPS (block)']].copy()
    
    pc.append(possible_calls)
    

0it [00:00, ?it/s]

In [13]:
possible_calls = pd.concat(pc)


In [16]:
orphan_flights_not_close.shape[0]

741

In [21]:
compiled_flights =[] 

for _, row in tqdm(orphan_flights_not_close.iterrows()):
    
    d = gpd.read_file(f"/Volumes/easystore/Drones/flights/kml/{row['id']}.kml",driver='KML')
    d['drone-id'] = row['id']
    d['address_map'] = row['address_map']
    d['drone-date'] = row['drone-date']
    d['incident_id'] = row['incident_id']
    d['type']=row['type']
     
    compiled_flights.append(d)
    

0it [00:00, ?it/s]

In [22]:
paths = pd.concat(compiled_flights,ignore_index=True)
paths = paths[paths['Name']!="Home"]


In [23]:
paths = paths[paths['geometry'].astype(str)!='LINESTRING EMPTY']


In [25]:
paths['drone-id'].drop_duplicates().shape[0]

586

In [26]:
orphan_flights_and_possible_calls = pd.merge(possible_calls,paths,on='drone-id',how='left')

In [30]:
coi = ['Incident No.', 'Date', 'Description_x', 'Priority', 'Disposition',
       'Call Source', 'Zip Code', 'Block Location', 'Case No', 'Address',
       'Latitude', 'Longitude', 'cfs-date', 'drone-id','drone-incident-id', 'drone-date_x', 'drone-type', 'drone-address','drone-FIPS', 'Name', 'Description_y', 'geometry', 'address_map','drone-date_y', 'incident_id', 'type']
orphan_flights_and_possible_calls.columns

Index(['Incident No.', 'Date', 'Description_x', 'Priority', 'Disposition',
       'Call Source', 'Zip Code', 'Block Location', 'Case No', 'Address',
       'Latitude', 'Longitude', 'Accuracy Score', 'Accuracy Type', 'Number',
       'Street', 'Unit Type', 'Unit Number', 'City', 'State', 'County', 'Zip',
       'Country', 'Source', 'Census Year', 'State FIPS', 'County FIPS',
       'Place Name', 'Place FIPS', 'Census Tract Code', 'Census Block Code',
       'Census Block Group', 'Full FIPS (block)', 'Full FIPS (tract)',
       'Metro/Micro Statistical Area Name',
       'Metro/Micro Statistical Area Code',
       'Metro/Micro Statistical Area Type', 'Combined Statistical Area Name',
       'Combined Statistical Area Code', 'Metropolitan Division Area Name',
       'Metropolitan Division Area Code', 'cfs-date', 'drone-id',
       'drone-incident-id', 'drone-date_x', 'drone-type', 'drone-address',
       'drone-FIPS', 'Name', 'Description_y', 'geometry', 'address_map',
       'drone-date_

In [50]:
mysteries = orphan_flights_and_possible_calls[(orphan_flights_and_possible_calls['type'].astype(str)=='nan')&(orphan_flights_and_possible_calls['incident_id'].astype(str)=='nan')]


477

In [53]:
sample=mysteries[['drone-id']].drop_duplicates().sample(100)

mysteries[mysteries['drone-id'].isin(sample['drone-id'])][coi].to_csv('./flights-n-calls.csv',index=False)

In [35]:
# a7df0277404f2e073ea25d556c93baeb

Incident No.                                                        230816L00070040
Date                                                        2023-08-16 15:22:47.237
Description                                                            Traffic Stop
Priority                                                                          5
Disposition                                                       HANDLED NO REPORT
Call Source                                                                 officer
Zip Code                                                                      91911
Block Location                                               FIRST AV & QUINTARD ST
Case No                                                                         NaN
Address                              FIRST AV & QUINTARD ST, Chula Vista, CA, 91911
Latitude                                                                   32.60562
Longitude                                                               -117

In [32]:
orphan_flights_and_possible_calls[coi]

,Incident No.,Date,Description_x,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,drone-type,drone-address,drone-FIPS,Name,Description_y,geometry,address_map,drone-date_y,incident_id,type
0,210728L00062790,2021-07-28 16:38:25.623,Ensure Safety of Fire and Medical Personnel,2,HANDLED NO REPORT,citizen,91911,1100 BLOCK THIRD AV,NaN,"1100 BLOCK THIRD AV, Chula Vista, CA, 91911",...,NaN,1710 Millenia Ave,060730133192021,Airdata.com,,"LINESTRING Z (-116.96935 32.61825 183.38220, -...",1710 Millenia Ave,2021-07-28 16:20:00,NaN,NaN
1,210728L00062795,2021-07-28 16:43:16.373,[Protected Call Type],4,HANDLED NO REPORT,citizen,91911,800 BLOCK RIVERLAWN AV,NaN,"800 BLOCK RIVERLAWN AV, Chula Vista, CA, 91911",...,NaN,1710 Millenia Ave,060730133192021,Airdata.com,,"LINESTRING Z (-116.96935 32.61825 183.38220, -...",1710 Millenia Ave,2021-07-28 16:20:00,NaN,NaN
2,210728L00062792,2021-07-28 16:39:07.763,[Protected Call Type],2,ASSISTED,citizen,91910,300 BLOCK E ST,2109450.0,"300 BLOCK E ST, Chula Vista, CA, 91910",...,NaN,1710 Millenia Ave,060730133192021,Airdata.com,,"LINESTRING Z (-116.96935 32.61825 183.38220, -...",1710 Millenia Ave,2021-07-28 16:20:00,NaN,NaN
3,210728L00062791,2021-07-28 16:40:09.317,Extra Patrol,5,HANDLED NO REPORT,officer,91910,700 BLOCK MARINA PW,NaN,"700 BLOCK MARINA PW, Chula Vista, CA, 91910",...,NaN,1710 Millenia Ave,060730133192021,Airdata.com,,"LINESTRING Z (-116.96935 32.61825 183.38220, -...",1710 Millenia Ave,2021-07-28 16:20:00,NaN,NaN
4,210728L00062782,2021-07-28 16:07:18.273,Vandalism,5,HANDLED NO REPORT,citizen,91910,E H ST & OTAY LAKES RD,NaN,"E H ST & OTAY LAKES RD, Chula Vista, CA, 91910",...,NaN,1710 Millenia Ave,060730133192021,Airdata.com,,"LINESTRING Z (-116.96935 32.61825 183.38220, -...",1710 Millenia Ave,2021-07-28 16:20:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7066,230818L00070714,2023-08-18 19:33:19.430,Disabled Vehicle,3,HANDLED NO REPORT,citizen,91913,OLYMPIC PW & LA MEDIA RD,NaN,"OLYMPIC PW & LA MEDIA RD, Chula Vista, CA, 91913",...,Unknown Problem,7 Naples St.,060730131024004,Airdata.com,,"LINESTRING Z (-117.08270 32.63997 22.14233, -1...",7 Naples St.,2023-08-18 20:03:00,CV2311196,Unknown Problem
7067,230818L00070721,2023-08-18 20:28:34.180,Assault,2,ASSISTED,citizen,91913,900 BLOCK OTAY LAKES RD,NaN,"900 BLOCK OTAY LAKES RD, Chula Vista, CA, 91913",...,Unknown Problem,7 Naples St.,060730131024004,Airdata.com,,"LINESTRING Z (-117.08270 32.63997 22.14233, -1...",7 Naples St.,2023-08-18 20:03:00,CV2311196,Unknown Problem
7068,230818L00070719,2023-08-18 20:15:49.507,Attempt Suicide,1,ASSISTED,citizen,91913,1700 BLOCK THOMPSON AV,2311197.0,"1700 BLOCK THOMPSON AV, Chula Vista, CA, 91913",...,Unknown Problem,7 Naples St.,060730131024004,Airdata.com,,"LINESTRING Z (-117.08270 32.63997 22.14233, -1...",7 Naples St.,2023-08-18 20:03:00,CV2311196,Unknown Problem
7069,230818L00070722,2023-08-18 20:31:11.220,Non-Injury Collision,2,CITATION,citizen,91910,800 BLOCK E H ST,NaN,"800 BLOCK E H ST, Chula Vista, CA, 91910",...,Unknown Problem,7 Naples St.,060730131024004,Airdata.com,,"LINESTRING Z (-117.08270 32.63997 22.14233, -1...",7 Naples St.,2023-08-18 20:03:00,CV2311196,Unknown Problem
